In [1]:
import pandas as pd
import numpy as np
import time
import sys
sys.path.append('tools/tensorflow-DeepFM-master/')

import config
import DataReader
from DeepFM import DeepFM




import tensorflow as tf

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import log_loss,roc_auc_score


<font color=#0099ff size=5 face="黑体">固定函数</font>

In [2]:
def hour_ouput(value):
    return time.localtime(value).tm_hour

def day_ouput(value):
    return time.localtime(value).tm_mday



def process(df):
    cols = [c for c in df.columns if c not in config.KEYS]
    for i in range(1,3):
        df['item_category_list_bin_%d'%i] = df['item_category_list'].apply(lambda x: x.split(';')[i] if len(x.split(';'))>i else -1)
        
        #df['item_property_list_bin_%d'%i] = df['item_property_list'].apply(lambda x: x.split(';')[i] if len(x.split(';'))>i else -1)
        #df['item_property_list_bin_%d'%i] = df['item_property_list'].apply(lambda x: x.split(';')[i] if len(x.split(';'))>i else -1)
    df['item_property_list'] = dfTrain['item_property_list'].apply(lambda x:';'.join(sorted(set(x.split(';')))))
    df["missing_feat"] = np.sum((df[cols] == -1).values, axis=1)
    df['context_timestamp_hour'] = df['context_timestamp'].map(hour_ouput)
    df['context_timestamp_day'] = df['context_timestamp'].map(day_ouput)
    return df

#def string_process(dfTrain,dfTest):
    

<font color=#0099ff size=5 face="黑体">读取数据</font>

In [3]:
dfTrain = pd.read_table(config.TRAIN_FILE,sep=' ')
dfTest = pd.read_table(config.TEST_FILE,sep=' ')

dfTrain = process(dfTrain)
dfTest = process(dfTest)


featInput = [c for c in dfTrain.columns if c not in config.KEYS]
featInput = [c for c in featInput if (not c in config.IGNORE_COLS)]

'''TrainX = dfTrain[featInput].values
TestX = dfTest[featInput].values
Trainy = dfTrain[config.LABEL].values'''

'TrainX = dfTrain[featInput].values\nTestX = dfTest[featInput].values\nTrainy = dfTrain[config.LABEL].values'

<font color=#0099ff size=5 face="黑体">生成稀疏特征</font>

In [4]:
fd = DataReader.FeatureDictionary(dfTrain=dfTrain, dfTest=dfTest,
                           numeric_cols=config.NUMERIC_COLS,
                           ignore_cols=config.IGNORE_COLS,
                           active_bound=1)
data_parser = DataReader.DataParser(feat_dict=fd)
Xi_train, Xv_train, y_train = data_parser.parse(df=dfTrain, has_label=True)
#Xi_test, Xv_test, ids_test = data_parser.parse(df=dfTest)

<font color=#0099ff size=5 face="黑体">设置模型参数</font>

In [8]:
dfm_params = {
    "use_fm": True,
    "use_deep": True,
    "embedding_size": 8,
    "dropout_fm": [1.0, 1.0],
    "deep_layers": [32, 32],
    "dropout_deep": [0.5, 0.5, 0.5],
    "deep_layers_activation": tf.nn.relu,
    "epoch": 20,
    "batch_size": 2048,
    "learning_rate": 0.1,
    "optimizer_type": "adam",
    "batch_norm": 1,
    "batch_norm_decay": 0.995,
    "l2_reg": 0.01,
    "verbose": True,
    "eval_metric": log_loss,
    "random_seed": config.RANDOM_SEED
}
dfm_params["feature_size"] = fd.feat_dim
dfm_params["field_size"] = len(Xi_train[0])
print(dfm_params["feature_size"])

22405


In [9]:
###根据日期拆分训练集
train_idx = dfTrain.loc[dfTrain['context_timestamp_day']<24].index
valid_idx = dfTrain.loc[dfTrain['context_timestamp_day']==24].index

#train_idx = list(train_idx)+appendList

_get = lambda x,l:[x[i] for i in l]
Xi_train_, Xv_train_, y_train_ = _get(Xi_train, train_idx), _get(Xv_train, train_idx), _get(y_train, train_idx)
Xi_valid_, Xv_valid_, y_valid_ = _get(Xi_train, valid_idx), _get(Xv_train, valid_idx), _get(y_train, valid_idx)

dfm = DeepFM(**dfm_params)
dfm.fit(Xi_train_, Xv_train_, y_train_, Xi_valid_, Xv_valid_, y_valid_)

#params: 208942
[1] train-result=0.0906, valid-result=0.0854 [20.4 s]
[2] train-result=0.0877, valid-result=0.0865 [20.6 s]
[3] train-result=0.0889, valid-result=0.0914 [20.4 s]
[4] train-result=0.0915, valid-result=0.0914 [21.3 s]
[5] train-result=0.0897, valid-result=0.0935 [21.0 s]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1694: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1694: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[6] train-result=nan, valid-result=0.0948 [20.4 s]
[7] train-result=0.0916, valid-result=0.0930 [20.6 s]
[8] train-result=0.0916, valid-result=0.0977 [20.9 s]
[9] train-result=inf, valid-result=0.0983 [20.2 s]
[10] train-result=0.0874, valid-result=0.0952 [20.4 s]
[11] train-result=0.0886, valid-result=0.0931 [22.4 s]
[12] train-result=0.0870, valid-result=0.0960 [20.3 s]
[13] train-result=0.0864, valid-result=0.1003 [20.5 s]
[14] train-result=0.0907, valid-result=0.1055 [20.7 s]
[15] train-result=0.0895, valid-result=0.0999 [20.7 s]
[16] train-result=0.0902, valid-result=0.0940 [20.6 s]
[17] train-result=inf, valid-result=0.1105 [20.6 s]
[18] train-result=nan, valid-result=inf [20.3 s]
[19] train-result=inf, valid-result=0.5841 [20.5 s]
[20] train-result=0.6609, valid-result=0.5841 [20.7 s]


In [ ]:
result_valid_ = dfm.predict( Xi_valid_, Xv_valid_)
print(log_loss(y_valid_, result_valid_))

In [ ]:
df = pd.DataFrame({'target':y_valid_,'score':result_valid_})
df['score'].max()

In [ ]:
df.sort_values('score',ascending=False,inplace=True)
df.reset_index(inplace=True)

In [ ]:
df.iloc[:20,1].mean()

In [ ]:
y_test_meta = np.zeros((dfTest.shape[0], 1), dtype=float)
y_test_meta[:,0] += dfm.predict(Xi_test, Xv_test)

In [ ]:
y_test_meta = np.zeros((dfTest.shape[0], 1), dtype=float)
y_test_meta[:,0] += dfm.predict(Xi_test, Xv_test)
y_test_meta[:,0].max()

In [ ]:
folds = list(KFold(n_splits=config.NUM_SPLITS,shuffle=True,random_state=config.RANDOM_SEED).split(dfTrain.values))
_get = lambda x,l:[x[i] for i in l]

y_test_meta = np.zeros((dfTest.shape[0], 1), dtype=float)

for i,(train_idx, valid_idx) in enumerate(folds):
    Xi_train_, Xv_train_, y_train_ = _get(Xi_train, train_idx), _get(Xv_train, train_idx), _get(y_train, train_idx)
    Xi_valid_, Xv_valid_, y_valid_ = _get(Xi_train, valid_idx), _get(Xv_train, valid_idx), _get(y_train, valid_idx)

    dfm = DeepFM(**dfm_params)
    dfm.fit(Xi_train_, Xv_train_, y_train_, Xi_valid_, Xv_valid_, y_valid_)
    
    y_test_meta[:,0] += dfm.predict(Xi_test, Xv_test)
    
y_test_meta /= float(len(folds))    

In [ ]:
submit = pd.DataFrame({'instance_id':ids_test,'predicted_score':y_test_meta[:,0]})
submit.to_csv('../../Submission/advertisement/FM_deep_0322.txt', sep=" ", index=False, line_terminator='\n')

In [ ]:
submit['predicted_score'].describe()

In [ ]:
np.log(0.9058)

In [ ]:
2.15/18371


In [ ]:
submit.shape